In [89]:
import tensorflow as tf
import keras

import os

from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam

from keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [90]:
img_shape = (48,48)
BS = 16
dir_images = 'data/Training'

train_ds = image_dataset_from_directory(
    dir_images,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=img_shape,
    batch_size=BS   
)

val_ds = image_dataset_from_directory(
    dir_images,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=img_shape,
    batch_size=BS   
)

Found 27463 files belonging to 7 classes.
Using 21971 files for training.
Found 27463 files belonging to 7 classes.
Using 5492 files for validation.


In [111]:
train_ds.target_size

AttributeError: 'BatchDataset' object has no attribute 'target_size'

In [106]:
shallow_sequence = [
    
    layers.Rescaling(1./255, input_shape=(img_shape+(1,))),
    
    #layer 1
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), #stride size = 1
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    #layer 2
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"), #stride size = 1
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),


    #FC layer
    layers.Flatten(),
    layers.Dense(512, activation='relu'),

    #logistic layer
    layers.Dense(num_cat, activation='softmax')

  ]

In [107]:
def facial_expr_model(sequence):

    model = keras.Sequential(sequence)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [108]:
model = facial_expr_model(shallow_sequence)
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 48, 48, 1)         0         
                                                                 
 conv2d_24 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 batch_normalization_24 (Bat  (None, 46, 46, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_24 (Dropout)        (None, 46, 46, 32)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 22, 22, 64)       0         
 g2D)                                                

In [109]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=4
)

Epoch 1/4


ValueError: in user code:

    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/jameskidd/opt/anaconda3/lib/python3.9/site-packages/keras/engine/input_spec.p